<h1><b>Players defensive stats scraping</h1></b>

In [3]:
#code inspiered by https://github.com/deeppomal/whoscored_scraper/blob/master/extraction%20using%20selenium.ipynb (the code is not really providing a proper table anymore)
#I also followed this youtube tutorial 
from selenium import webdriver 
from selenium.webdriver.common.keys import Keys #import keys to navigate the web page
from selenium.common.exceptions import TimeoutException, WebDriverException
import time

import pandas as pd
import re
import os

<h3><b>Selenium webdriver</h3></b>

>Here below I report all the web pages link used for the scraping. As mentioned above, this do not loop across all the leagues but has to be manually changed based on the searched league i.e. you must copy the selected url variable from the cell below and copy it at 'website_URL'.

In [53]:
Premier_league = 'https://www.whoscored.com/Regions/252/Tournaments/2/Seasons/8228/Stages/18685/PlayerStatistics/England-Premier-League-2020-2021'
Serie_a = 'https://www.whoscored.com/Regions/108/Tournaments/5/Seasons/8330/Stages/18873/PlayerStatistics/Italy-Serie-A-2020-2021'
La_Liga = 'https://www.whoscored.com/Regions/206/Tournaments/4/Seasons/8321/Stages/18851/PlayerStatistics/Spain-LaLiga-2020-2021'
Bundesliga = 'https://www.whoscored.com/Regions/81/Tournaments/3/Seasons/8279/Stages/18762/PlayerStatistics/Germany-Bundesliga-2020-2021'
Ligue_1 = 'https://www.whoscored.com/Regions/74/Tournaments/22/Seasons/8185/Stages/18594/PlayerStatistics/France-Ligue-1-2020-2021'
Liga_nos = 'https://www.whoscored.com/Regions/177/Tournaments/21/Seasons/8315/Stages/18842/PlayerStatistics/Portugal-Liga-NOS-2020-2021'
Eredivisie = 'https://www.whoscored.com/Regions/155/Tournaments/13/Seasons/8187/Stages/18596/PlayerStatistics/Netherlands-Eredivisie-2020-2021'
Russian_premier_league = 'https://www.whoscored.com/Regions/182/Tournaments/77/Seasons/8242/Stages/18707/PlayerStatistics/Russia-Premier-League-2020-2021'
Brasileirao = 'https://www.whoscored.com/Regions/31/Tournaments/95/Seasons/8555/Stages/19551/PlayerStatistics/Brazil-Brasileir%C3%A3o-2021'
MLS = 'https://www.whoscored.com/Regions/233/Tournaments/85/Seasons/8545/Stages/19513/PlayerStatistics/USA-Major-League-Soccer-2021'
Super_lig = 'https://www.whoscored.com/Regions/225/Tournaments/17/Seasons/8313/Stages/18838/PlayerStatistics/Turkey-Super-Lig-2020-2021'
Championship = 'https://www.whoscored.com/Regions/252/Tournaments/7/Seasons/8304/Stages/19661/PlayerStatistics/England-Championship-2020-2021'
Premiership = 'https://www.whoscored.com/Regions/253/Tournaments/20/Seasons/8191/Stages/19535/PlayerStatistics/Scotland-Premiership-2020-2021'
League_one = 'https://www.whoscored.com/Regions/252/Tournaments/8/Seasons/8305/Stages/19705/PlayerStatistics/England-League-One-2020-2021'
League_two = 'https://www.whoscored.com/Regions/252/Tournaments/9/Seasons/8306/Stages/19704/PlayerStatistics/England-League-Two-2020-2021'
Superliga = 'https://www.whoscored.com/Regions/11/Tournaments/68/Seasons/7905/Stages/17737/PlayerStatistics/Argentina-Superliga-2019-2020'#season 2021 starts on 18 july 2021
Jupiler_league = 'https://www.whoscored.com/Regions/22/Tournaments/18/Seasons/8197/Stages/19624/PlayerStatistics/Belgium-Jupiler-Pro-League-2020-2021'
Bundesliga_2 = 'https://www.whoscored.com/Regions/81/Tournaments/6/Seasons/8280/Stages/18763/PlayerStatistics/Germany-Bundesliga-II-2020-2021'

In [312]:
#define browser
driver = webdriver.Chrome('chromedriver.exe')#for this scraping I'm using chrome as browser

#define URL
website_URL = Bundesliga_2
driver.get(website_URL)

#by default, the data are filtered. I need to click on the button to make all the players available
driver.find_element_by_link_text('Defensive').click()
driver.find_element_by_link_text('All players').click()

In [313]:
#set page number
#number of pages e.g premier league it's 54 so I need to set max_page as 54
page = 1
max_page= 51

#max page for each league
#Premier league 54
#Serie A 63
#La liga 59
#Bundensilga 51
#Ligue 1 59
#Liga NOS 55
#Eredivisie 53
#Russian premier league 53
#Brasileirão 31
#MLS 59
#Super lig 70
#Championship 74
#Premiership 37
#League one 75
#League two 74
#Superliga 73
#Jupiler Pro league 55
#Bundensliga_2 51

In [314]:
data = []

while True:
        try:
            if page > max_page :
                print('Last page reached')
                break
            page+=1
            for i in driver.find_elements_by_id('statistics-table-defensive'): #define the html target
                table = driver.find_element_by_id('statistics-table-defensive').text #note _elements is different from _element
                data.append(table)
            driver.find_element_by_link_text('next').click() #move to the next page
            time.sleep(2) #put the program to sleep for 2 seconds i.e. enough time to ensure that the following page is upload before execution
            print('Page', page-1) #print pages computed (double check that is = to max_page)
        except (TimeoutException, WebDriverException) as e:
            print("Done")
            break
            
driver.quit() #terminate the session

Page 1
Page 2
Page 3
Page 4
Page 5
Page 6
Page 7
Page 8
Page 9
Page 10
Page 11
Page 12
Page 13
Page 14
Page 15
Page 16
Page 17
Page 18
Page 19
Page 20
Page 21
Page 22
Page 23
Page 24
Page 25
Page 26
Page 27
Page 28
Page 29
Page 30
Page 31
Page 32
Page 33
Page 34
Page 35
Page 36
Page 37
Page 38
Page 39
Page 40
Page 41
Page 42
Page 43
Page 44
Page 45
Page 46
Page 47
Page 48
Page 49
Page 50
Page 51
Last page reached


<h3><b>Data Manipulation</h3></b>

> Here below I turn the extracted data into a readable format

In [315]:
# here I need to turn the data in a suitable for manipulation 
df = '.'.join(str(x) for x in data) #this is necessary to turn the list into str (https://stackoverflow.com/questions/37377389/convert-list-into-string-in-python3)
df = re.split ('\n', df) #split based on the '\n' pattern
df = pd.DataFrame(df) #turn the data to pd to ease the manipulation
df.columns = ['col'] #rename the column (there is only 1 now)

>note: on each page the table headers are repeated and so in the df. By printing df, the structure is
>- header (player) -> this is repeated every time the selenium script read ('next').click()
>- stats headers (Apps Mins Goals Assists Yel Red SpG PS% Aerial...) -> this is repeated every time the selenium script read ('next').click()
>- player number in the list
>- team, age roles
>- stats

In [316]:
# here I extract some of the headers that I need to compose the final df before deleting those
headers = df[1:2] # stats are located in the second row
headers = headers.col.str.split(' ',expand=True) #split

headers

,0,1,2,3,4,5,6,7,8,9,10
1,Apps,Mins,Tackles,Inter,Fouls,Offsides,Clear,Drb,Blocks,OwnG,Rating


In [317]:
# remove headers from the dataframe
df = df[~df.col.str.contains('Apps Mins Tackles Inter Fouls')] #this removes entires rows
df['col'] = df['col'].str.replace('.Player','') #replace with blank space Player

In [318]:
# here I must 
body = df.iloc[1:] #save the table body removing the first two rows (not empty because 'Player' has been removed)
body = pd.DataFrame(body.col.values.reshape(-1, 4)) #reshaping the dataframe will arrange the data in the proper format
body.columns = ['col0', 'col1', 'col2', 'col3'] #rename columns

body.head(5)

,col0,col1,col2,col3
0,1,Philipp Klewin,"Erzgebirge Aue, 27, Goalkeeper",1 90 - - - - - - - - 7.98
1,2,Robert Zulj,"Bochum, 29, AM(C),FW",30(1) 2634 0.8 0.4 1.6 - 1 0.6 0.1 - 7.67
2,3,Tim Kleindienst,"FC Heidenheim, 25, FW",15 1276 0.5 0.7 2.9 - 1.6 1 0.1 - 7.54
3,4,Serdar Dursun,"Darmstadt, 29, FW",32(1) 2695 0.4 0.2 1.7 - 1.2 0.4 0.1 - 7.45
4,5,Philipp Hofmann,"Karlsruher SC, 28, FW",30 2628 0.4 0.1 1.6 0.1 0.8 0.4 0.1 - 7.43


In [319]:
body_1 = body.col1 #player name 
body_2 = body.col2.str.split(',',expand=True) #split col 2
body_3 = body.col3.str.split(' ',expand=True) #split col 3
body = pd.concat([body_1, body_2, body_3], axis=1) #concat data
body = body.iloc[:, :-1] #remove last columns (report the rating which is already in summary)

body.head(5)

,col1,0,1,2,3,4,0,1,2,3,4,5,6,7,8,9
0,Philipp Klewin,Erzgebirge Aue,27,Goalkeeper,None,None,1,90,-,-,-,-,-,-,-,-
1,Robert Zulj,Bochum,29,AM(C),FW,None,30(1),2634,0.8,0.4,1.6,-,1,0.6,0.1,-
2,Tim Kleindienst,FC Heidenheim,25,FW,None,None,15,1276,0.5,0.7,2.9,-,1.6,1,0.1,-
3,Serdar Dursun,Darmstadt,29,FW,None,None,32(1),2695,0.4,0.2,1.7,-,1.2,0.4,0.1,-
4,Philipp Hofmann,Karlsruher SC,28,FW,None,None,30,2628,0.4,0.1,1.6,0.1,0.8,0.4,0.1,-


In [320]:
#Premiership has not third_role. Add an empy column 
#body.insert(loc=5, column = 'col', value=['' for i in range(body.shape[0])])

In [321]:
#df clean
df_clean = body.set_axis(['Name', 'Team', 'Age', 'Main_role', 'Secondary_role',
                          'third_role', 'Apps', 'Mins', 'Tackles', 'Inter',
                          'Fouls', 'Offsides', 'Clear', 'Drb', 'Blocks','OwnG'], axis=1, inplace=False)

df_clean.head(5)

,Name,Team,Age,Main_role,Secondary_role,third_role,Apps,Mins,Tackles,Inter,Fouls,Offsides,Clear,Drb,Blocks,OwnG
0,Philipp Klewin,Erzgebirge Aue,27,Goalkeeper,None,None,1,90,-,-,-,-,-,-,-,-
1,Robert Zulj,Bochum,29,AM(C),FW,None,30(1),2634,0.8,0.4,1.6,-,1,0.6,0.1,-
2,Tim Kleindienst,FC Heidenheim,25,FW,None,None,15,1276,0.5,0.7,2.9,-,1.6,1,0.1,-
3,Serdar Dursun,Darmstadt,29,FW,None,None,32(1),2695,0.4,0.2,1.7,-,1.2,0.4,0.1,-
4,Philipp Hofmann,Karlsruher SC,28,FW,None,None,30,2628,0.4,0.1,1.6,0.1,0.8,0.4,0.1,-


In [322]:
#drop columns that are repeated in the summary (name is retained to ensure a correspodance across data frames)
df_clean.drop(['Team', 'Age', 'Main_role', 'Secondary_role', 'third_role', 'Apps', 'Mins'], axis=1, inplace=True)

In [323]:
df_clean.shape

(504, 9)

In [324]:
#by doing this I noticed that sometimes there is some sort of bug in selenium that produces duplicates but is not always reproducible
df_clean.loc[df_clean.duplicated(),:]

,Name,Tackles,Inter,Fouls,Offsides,Clear,Drb,Blocks,OwnG


In [325]:
#save as csv
df_clean.to_csv('Bundesliga_2_defensive.csv', index=False)

<h3><b>Reload all the dataframes</h3></b>

In [35]:
#while uploading the dataframes I will also add a column to specify the league where they are playing
Premier_league = pd.read_csv(os.path.join('Premier_league_defensive.csv')) 
Premier_league['League'] = 'Premier_league'

Serie_a = pd.read_csv(os.path.join('Serie_a_defensive.csv')) 
Serie_a['League'] = 'Serie_A'

La_Liga = pd.read_csv(os.path.join('La_Liga_defensive.csv')) 
La_Liga['League'] = 'La_Liga'

Bundesliga = pd.read_csv(os.path.join('Bundesliga_defensive.csv')) 
Bundesliga['League'] = 'Bundesliga'

Ligue_1 = pd.read_csv(os.path.join('Ligue_1_defensive.csv')) 
Ligue_1['League'] = 'Ligue_1'

Liga_nos = pd.read_csv(os.path.join('Liga_nos_defensive.csv')) 
Liga_nos['League'] = 'Liga_nos'

Eredivisie = pd.read_csv(os.path.join('Eredivisie_defensive.csv')) 
Eredivisie['League'] = 'Eredivisie'

Russian_premier_league = pd.read_csv(os.path.join('Russian_premier_league_defensive.csv')) 
Russian_premier_league['League'] = 'Russian_premier_league'

Brasileirao = pd.read_csv(os.path.join('Brasileirao_defensive.csv')) 
Brasileirao['League'] = 'Brasileirao'

MLS = pd.read_csv(os.path.join('MLS_defensive.csv'))
MLS['League'] = 'MLS'

Super_lig = pd.read_csv(os.path.join('Super_lig_defensive.csv')) 
Super_lig['League'] = 'Super_lig'

Championship = pd.read_csv(os.path.join('Championship_defensive.csv')) 
Championship['League'] = 'Championship'

Premiership = pd.read_csv(os.path.join('Premiership_defensive.csv')) 
Premiership['League'] = 'Premiership'

League_one = pd.read_csv(os.path.join('League_one_defensive.csv'))
League_one['League'] = 'League_one'

League_two = pd.read_csv(os.path.join('League_two_defensive.csv')) 
League_two['League'] = 'League_two'

Superliga = pd.read_csv(os.path.join('Superliga_defensive.csv')) 
Superliga['League'] = 'Superliga'

Jupiler_league = pd.read_csv(os.path.join('Jupiler_league_defensive.csv')) 
Jupiler_league['League'] = 'Jupiler_league'

Bundesliga_2 = pd.read_csv(os.path.join('Bundesliga_2_defensive.csv')) 
Bundesliga_2['League'] = 'Bundesliga_2'

In [36]:
stats_def = pd.concat([Premier_league, Serie_a, La_Liga, Bundesliga, Ligue_1,
                       Liga_nos, Eredivisie, Russian_premier_league, Brasileirao, MLS,
                       Super_lig, Championship, Premiership, League_one, League_two,
                       Superliga, Jupiler_league, Bundesliga_2])
stats_def.shape

(10356, 10)

In [37]:
stats_def.head(5)

,Name,Tackles,Inter,Fouls,Offsides,Clear,Drb,Blocks,OwnG,League
0,Harry Kane,0.6,0.3,0.7,-,1.1,0.5,0.3,-,Premier_league
1,Kevin De Bruyne,1.4,0.4,1.1,-,0.3,1.4,-,-,Premier_league
2,Jack Grealish,1.2,0.5,1.5,-,0.2,0.3,0.1,-,Premier_league
3,Bruno Fernandes,1.5,0.7,1.2,-,0.7,2,-,-,Premier_league
4,Tomas Soucek,2.1,1.6,1.8,0.1,2.7,1.3,0.5,1,Premier_league


In [38]:
stats_def.loc[stats_def.duplicated(),:]

,Name,Tackles,Inter,Fouls,Offsides,Clear,Drb,Blocks,OwnG,League


In [39]:
#save as csv
stats_def.to_csv('stats_defensive.csv', index=False)